# **Prueba Técnica Data Analyst**

## Análisis de Datos con SQL y Python (Pandas)

**Contexto:**
La empresa *BCA Operations* requiere analizar los datos de sus jugadores para:
- Identificar jugadores con **primer depósito** (FTD).
- Filtrar jugadores que cumplen con la condición de **CPA** (depósitos totales > 100 USD).
- Comparar los resultados con los registros internos para encontrar discrepancias.
- Visualizar información relevante de depósitos y jugadores.

Este Notebook sigue los pasos indicados en la prueba técnica y está cuidadosamente comentado para explicar la lógica y la metodología aplicada.

### **Objetivos Principales**
1. **SQL**: Crear una base de datos y tablas a partir de CSV. Realizar consultas para:
   - Identificar FTD.
   - Filtrar CPA.
   - Generar reporte de FTD y CPA por mes.
2. **Python (Pandas)**: Comparar con `internal_records.csv` y detectar discrepancias.
3. **Visualizaciones**: Graficar discrepancias, histórico de depósitos y otros insights.


In [18]:
# ==============
# 1. IMPORTACIONES
# ==============
# En esta celda realizamos la importación de todas las librerías necesarias.
# Se recomienda centralizar las importaciones para facilitar la lectura y el mantenimiento.

import sqlite3  # Para crear y gestionar la base de datos SQLite
import pandas as pd  # Para manipulación y análisis de datos
import numpy as np  # Para operaciones numéricas y manejo de arreglos
import matplotlib.pyplot as plt  # Para generación de gráficos en Python
import seaborn as sns  # Para gráficos estadísticos (facilita la visualización)

# Configuración de estilo para gráficos
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (10, 5)  # Define el tamaño de los gráficos

## 2. **Carga de Datos**
En esta sección:
- Leemos los archivos CSV una sola vez.
- Convertimos las fechas a tipo datetime.
- Mostramos una vista previa para verificar la correcta carga de datos.

In [ ]:
# Cargar datos en DataFrames de pandas.
# Se asume que los archivos se encuentran en la carpeta 'data'.
# Ajusta la ruta en caso de que estén en otra ubicación.

df_players = pd.read_csv("./data/players.csv")
df_traffic_sources = pd.read_csv("./data/traffic_sources.csv")
df_deposits = pd.read_csv("./data/deposits.csv")
df_internal_records = pd.read_csv("./data/internal_records.csv")

# Convertir columnas de fecha a tipo datetime (evita errores en operaciones de fechas)
df_players["registration_date"] = pd.to_datetime(df_players["registration_date"], errors='coerce')
df_traffic_sources["registration_date"] = pd.to_datetime(df_traffic_sources["registration_date"], errors='coerce')
df_deposits["deposit_date"] = pd.to_datetime(df_deposits["deposit_date"], errors='coerce')
df_internal_records["month"] = pd.to_datetime(df_internal_records["month"], errors='coerce')

print("Datos cargados y convertidos correctamente.")

# Muestra de los primeros registros de cada DataFrame
display(df_players.head())
display(df_traffic_sources.head())
display(df_deposits.head())
display(df_internal_records.head())

## 3. **Creación de la Base de Datos y Tablas SQL**
En esta sección:
1. Conectamos/creamos la base de datos `gaming_data.db` con **SQLite**.
2. Definimos las tablas necesarias: `players`, `traffic_sources`, `deposits`, `internal_records`.
3. Insertamos los datos directamente desde los DataFrames a la base de datos.

In [ ]:
# Conectar a la base de datos (se creará si no existe)
conn = sqlite3.connect("gaming_data.db")
cursor = conn.cursor()

# Creamos las tablas necesarias si no existen.
# Usamos la instrucción EXECUTESCRIPT para ejecutar múltiples comandos SQL.

cursor.executescript('''
    CREATE TABLE IF NOT EXISTS players (
        player_id INTEGER PRIMARY KEY,
        name TEXT,
        country TEXT,
        registration_date DATE
    );
    
    CREATE TABLE IF NOT EXISTS traffic_sources (
        player_id INTEGER,
        trafficker TEXT,
        cost_of_acquisition REAL,
        registration_date DATE,
        source TEXT
    );
    
    CREATE TABLE IF NOT EXISTS deposits (
        deposit_id INTEGER PRIMARY KEY,
        player_id INTEGER,
        deposit_date DATE,
        deposit_amount REAL
    );
    
    CREATE TABLE IF NOT EXISTS internal_records (
        month DATE,
        expected_ftd INTEGER,
        expected_cpa INTEGER,
        status TEXT
    );
''')

conn.commit()
print("Tablas creadas o verificadas correctamente.")

# Cargamos los datos de los DataFrames a las tablas de SQLite.
# if_exists='replace' indica que se reemplazará la tabla si ya existe contenido.

df_players.to_sql("players", conn, if_exists="replace", index=False)
df_traffic_sources.to_sql("traffic_sources", conn, if_exists="replace", index=False)
df_deposits.to_sql("deposits", conn, if_exists="replace", index=False)
df_internal_records.to_sql("internal_records", conn, if_exists="replace", index=False)

print("Datos insertados en la base de datos correctamente.")

## 4. **Identificar el Primer Depósito (FTD)**
El **FTD (First Time Deposit)** se define como la fecha mínima de depósito por jugador.
- Creamos una tabla auxiliar `first_time_deposits` para almacenar el `player_id` y la fecha de su primer depósito.

In [ ]:
# Eliminamos la tabla si ya existe, para recalcular
cursor.execute("DROP TABLE IF EXISTS first_time_deposits;")
conn.commit()

# Consulta para crear la tabla con el primer depósito de cada jugador (fecha mínima)
cursor.execute('''
    CREATE TABLE first_time_deposits AS
    SELECT
        player_id,
        MIN(deposit_date) AS first_deposit_date
    FROM deposits
    GROUP BY player_id;
''')
conn.commit()

print("Tabla 'first_time_deposits' creada correctamente (identifica FTD).")

## 5. **Filtrar Jugadores CPA**
Según la prueba técnica, un jugador se considera **CPA** si la suma de sus depósitos supera los **100 USD** (o el valor indicado).

In [ ]:
# Eliminamos la tabla si existe, para recalcular
cursor.execute("DROP TABLE IF EXISTS cpa_players;")
conn.commit()

# Creamos la tabla con los jugadores que superan un total de 100 USD en depósitos
cursor.execute('''
    CREATE TABLE cpa_players AS
    SELECT
        player_id,
        SUM(deposit_amount) AS total_deposits
    FROM deposits
    GROUP BY player_id
    HAVING total_deposits > 100;
''')
conn.commit()

print("Tabla 'cpa_players' creada correctamente (jugadores con depósitos > 100 USD).")

## 6. **Reporte de FTD por Mes**
Generamos un reporte con la **cantidad de jugadores** que realizaron su primer depósito **cada mes**.

In [ ]:
# Consulta SQL para obtener el conteo de FTD por mes (usando strftime('%Y-%m', fecha))

query_ftd_month = '''
SELECT 
    strftime('%Y-%m', first_deposit_date) AS month,
    COUNT(player_id) AS ftd_count
FROM first_time_deposits
GROUP BY month
ORDER BY month;
'''

# Ejecutamos la consulta y convertimos a DataFrame
df_ftd_report = pd.read_sql(query_ftd_month, conn)
print("Reporte de FTD por mes:")
display(df_ftd_report)

## 7. **Reporte de CPA por Mes**
Generamos un reporte con la **cantidad de jugadores** que cumplen la condición de CPA **cada mes**.

In [ ]:
# Consulta SQL para obtener el conteo de CPA por mes.
# Se utiliza la tabla cpa_players en JOIN con deposits para extraer la fecha.

query_cpa_month = '''
SELECT 
    strftime('%Y-%m', d.deposit_date) AS month,
    COUNT(DISTINCT d.player_id) AS cpa_count
FROM deposits d
JOIN cpa_players c ON d.player_id = c.player_id
GROUP BY month
ORDER BY month;
'''

# Ejecutamos la consulta y lo llevamos a un DataFrame
df_cpa_report = pd.read_sql(query_cpa_month, conn)
print("Reporte de CPA por mes:")
display(df_cpa_report)

## 8. **Análisis de Calidad de Datos**
En esta sección:
1. Comparamos los resultados de FTD y CPA que calculamos con `internal_records.csv`.
2. Buscamos **discrepancias** por mes entre nuestros datos y los registros internos.
3. Guardamos en un DataFrame las diferencias detectadas.

In [ ]:
# Convertir la columna "month" a tipo datetime en todos los DataFrames
df_ftd_report["month"] = pd.to_datetime(df_ftd_report["month"], errors='coerce')
df_cpa_report["month"] = pd.to_datetime(df_cpa_report["month"], errors='coerce')
df_internal_records["month"] = pd.to_datetime(df_internal_records["month"], errors='coerce')

# Convertir la fecha a periodo (AÑO-MES) para hacer el merge correctamente
df_ftd_report["month_period"] = df_ftd_report["month"].dt.to_period("M")
df_cpa_report["month_period"] = df_cpa_report["month"].dt.to_period("M")
df_internal_records["month_period"] = df_internal_records["month"].dt.to_period("M")

# Unimos el reporte de FTD con el DataFrame de internal_records
df_comparison = df_internal_records.merge(
    df_ftd_report[["month_period", "ftd_count"]],
    how="left",
    on="month_period"
)

# Unimos el reporte de CPA
df_comparison = df_comparison.merge(
    df_cpa_report[["month_period", "cpa_count"]],
    how="left",
    on="month_period"
)

# Renombramos columnas para unificar criterio
df_comparison.rename(columns={
    "expected_ftd": "ftd_expected",
    "expected_cpa": "cpa_expected"
}, inplace=True)

# Rellenamos NaN con 0 para evitar problemas de comparación
df_comparison[["ftd_count", "cpa_count"]] = df_comparison[["ftd_count", "cpa_count"]].fillna(0)

# Calculamos discrepancias
df_comparison["ftd_discrepancy"] = df_comparison["ftd_count"] - df_comparison["ftd_expected"]
df_comparison["cpa_discrepancy"] = df_comparison["cpa_count"] - df_comparison["cpa_expected"]

print("Discrepancias FTD y CPA vs. registros internos:")
display(df_comparison)

### 8.1 Visualización de Discrepancias
Graficamos las diferencias en **FTD** y **CPA** por mes para detectar patrones de error.

In [ ]:
# Filtramos los meses con discrepancias distintas de 0
df_discrepant = df_comparison[
    (df_comparison["ftd_discrepancy"] != 0) | (df_comparison["cpa_discrepancy"] != 0)
]

if not df_discrepant.empty:
    # Configuramos la figura con 2 subplots en la misma fila
    fig, ax = plt.subplots(1, 2, figsize=(14, 5))

    # Subplot 1: discrepancias FTD
    ax[0].bar(
        df_discrepant["month_period"].astype(str),
        df_discrepant["ftd_discrepancy"],
        color='coral', alpha=0.7
    )
    ax[0].axhline(0, color='black', linestyle='--', linewidth=1)
    ax[0].set_title("Discrepancias en FTD")
    ax[0].set_xlabel("Mes")
    ax[0].set_ylabel("(FTD Calculado) - (FTD Esperado)")
    ax[0].tick_params(axis='x', rotation=45)

    # Subplot 2: discrepancias CPA
    ax[1].bar(
        df_discrepant["month_period"].astype(str),
        df_discrepant["cpa_discrepancy"],
        color='skyblue', alpha=0.7
    )
    ax[1].axhline(0, color='black', linestyle='--', linewidth=1)
    ax[1].set_title("Discrepancias en CPA")
    ax[1].set_xlabel("Mes")
    ax[1].set_ylabel("(CPA Calculado) - (CPA Esperado)")
    ax[1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()
else:
    print("No se encontraron discrepancias en los datos para graficar.")

## 9. **Análisis de Depósitos**
En esta parte:
- Mostramos el histórico de dinero depositado mes a mes.
- Identificamos el jugador que más depositó.
- Realizamos una visualización de la distribución de depósitos por jugador.


In [ ]:
# Convertimos la columna deposit_amount a numérico por si hay valores inválidos.
df_deposits["deposit_amount"] = pd.to_numeric(df_deposits["deposit_amount"], errors='coerce')

# Creamos la columna 'month_year' (periodo) para agrupar los depósitos mensualmente
df_deposits["month_year"] = df_deposits["deposit_date"].dt.to_period("M")

# Calculamos el total de depósitos por mes
df_monthly_deposits = df_deposits.groupby("month_year")["deposit_amount"].sum().reset_index()

# Identificamos al jugador con mayor monto total depositado
df_deposits_by_player = df_deposits.groupby("player_id")["deposit_amount"].sum().reset_index()
max_deposit_row = df_deposits_by_player.loc[df_deposits_by_player["deposit_amount"].idxmax()]
player_with_max = max_deposit_row["player_id"]
max_deposit_amount = max_deposit_row["deposit_amount"]

print(f"Jugador con más depósitos: ID {player_with_max} con un total de ${max_deposit_amount:.2f}.")

print("\nTotal de depósitos por mes:")
display(df_monthly_deposits)

### 9.1 **Visualizaciones de Depósitos**
1. **Histórico de dinero depositado**: línea temporal.
2. **Distribución de depósitos por jugador**: histograma.
3. **Impacto de fuentes de tráfico** en el total de depósitos.

In [ ]:
# 1. Gráfico del histórico de dinero depositado por mes
plt.figure(figsize=(10, 4))
plt.plot(
    df_monthly_deposits["month_year"].astype(str),
    df_monthly_deposits["deposit_amount"],
    marker='o',
    linestyle='-'
)
plt.xticks(rotation=45)
plt.xlabel("Mes")
plt.ylabel("Total Depositado ($)")
plt.title("Histórico de Dinero Depositado por Mes")
plt.grid(True)
plt.show()

# 2. Distribución de depósitos por jugador
plt.figure(figsize=(10, 4))
sns.histplot(df_deposits_by_player["deposit_amount"], bins=30, kde=True, color='green')
plt.xlabel("Total Depositado por Jugador ($)")
plt.ylabel("Cantidad de Jugadores")
plt.title("Distribución de Depósitos por Jugador")
plt.show()

# 3. Impacto de las fuentes de tráfico (trafficker) en el total de depósitos
# Unir los DataFrames: df_deposits y df_traffic_sources para ver la relación

df_merged = df_deposits.merge(df_traffic_sources, on="player_id", how="left", suffixes=("_dep", "_ts"))
df_traffic_deposits = df_merged.groupby("trafficker")["deposit_amount"].sum().reset_index()

plt.figure(figsize=(10, 4))
sns.barplot(data=df_traffic_deposits, x="trafficker", y="deposit_amount", color='blue')
plt.xticks(rotation=45)
plt.xlabel("Trafficker")
plt.ylabel("Total Depositado ($)")
plt.title("Total de Depósitos por Trafficker")
plt.show()

# **Conclusiones y Próximos Pasos**
1. **FTD y CPA**: Hemos identificado correctamente los jugadores que hicieron su primer depósito (FTD) y aquellos que superan los 100 USD en total depositado (CPA).
2. **Discrepancias**: Se compararon los valores calculados con la tabla interna `internal_records`. Los meses con diferencias se muestran en los gráficos de discrepancias.
3. **Análisis de Depósitos**: Observamos el histórico mensual, el jugador de máximo depósito y la distribución general. También podemos ver qué **traffickers** generan mayores volúmenes de depósito.
4. **Próximos pasos**:
   - Integrar estos hallazgos en **Power BI** para un dashboard más completo.
   - Validar con mayor detalle los meses con discrepancias.
   - Ajustar y optimizar los costos de adquisición si se identifican fuentes de tráfico poco rentables.

### **Fin de la Prueba Técnica**
Este Notebook cumple con los objetivos solicitados:
- Creación y carga de datos en SQLite.
- Identificación y reporte de FTD y CPA.
- Comparación con registros internos y detección de discrepancias.
- Visualizaciones clave de depósitos.

¡Gracias por tu atención!